# Load crypto market data

In [2]:
!git clone https://github.com/binance/binance-public-data.git

Cloning into 'binance-public-data'...
remote: Enumerating objects: 265, done.
remote: Counting objects: 100% (182/182), done.
remote: Compressing objects: 100% (124/124), done.
remote: Total 265 (delta 110), reused 110 (delta 58), pack-reused 83
Receiving objects: 100% (265/265), 6.50 MiB | 30.54 MiB/s, done.
Resolving deltas: 100% (138/138), done.


In [3]:
fromDate='2017-08-01'
toDate='2022-08-31'

In [4]:
!python3 binance-public-data/python/download-kline.py -t spot -s BTCUSDT -i 1m -startDate 2017-08-01 -endDate 2022-08-31 -skip-daily 1 -folder ~/marketdata/btc/
# !python3 binance-public-data/python/download-kline.py -t spot -s ETHUSDT -i 1m -startDate 2017-08-01 -endDate 2022-08-31 -skip-daily 1 -folder ~/marketdata/eth/

Found 1 symbols
[1/1] - start download monthly BTCUSDT klines 

File Download: /home/marco/marketdata/btc/data/spot/monthly/klines/BTCUSDT/1m/2017-08-01_2022-08-31/BTCUSDT-1m-2017-08.zip
[##################################################]
File Download: /home/marco/marketdata/btc/data/spot/monthly/klines/BTCUSDT/1m/2017-08-01_2022-08-31/BTCUSDT-1m-2017-09.zip
[##################################################]
File Download: /home/marco/marketdata/btc/data/spot/monthly/klines/BTCUSDT/1m/2017-08-01_2022-08-31/BTCUSDT-1m-2017-10.zip
[##################################################]
File Download: /home/marco/marketdata/btc/data/spot/monthly/klines/BTCUSDT/1m/2017-08-01_2022-08-31/BTCUSDT-1m-2017-11.zip
[##################################################]
File Download: /home/marco/marketdata/btc/data/spot/monthly/klines/BTCUSDT/1m/2017-08-01_2022-08-31/BTCUSDT-1m-2017-12.zip
[##################################################]
File Download: /home/marco/marketdata/btc/data/spot/mont

In [8]:
import zipfile, os

def unzipFiles(sym,ticker):
    working_directory = '/marketdata/'+sym+'/data/spot/monthly/klines/'+ticker+'/1m/'+fromDate+'_'+toDate+'/'
    for file in os.listdir(working_directory):   # get the list of files
        print(file)
        if zipfile.is_zipfile(working_directory+file): # if it is a zipfile, extract it
            with zipfile.ZipFile(working_directory+file) as item: # treat the file as a zip
                print(item)
                item.extractall('marketdata/'+sym)

unzipFiles('btc','BTCUSDT')
# unzipFiles('eth','ETHUSDT')

PermissionError: [Errno 13] Permission denied: '/root/marketdata/btc/data/spot/monthly/klines/BTCUSDT/1m/2017-08-01_2022-08-31/'

In [ ]:
S3_BUCKET=!(aws cloudformation list-exports --query "Exports[?Name=='public-blockchain-marketdata'].Value" --output text)
S3_BUCKET=S3_BUCKET[0]

In [ ]:
!pip install awswrangler

In [ ]:
import zipfile, os
import boto3
import pandas as pd
import awswrangler as wr
from datetime import datetime

s3_path='s3://'+S3_BUCKET+"/marketdata/crypto"

def importFiles(sym,ticker):
    for file in os.listdir("marketdata/"+sym+"/"):   # get the list of files
        if file.endswith(".csv"): # check for ".zip" extension
            file_name = 'marketdata/'+sym+'/'+file # get full path of files
            #print(file_name)
            x=file_name.split('/')[-1:][0].split(".")[0]
            #print(x)
            
            hData=['open_time','open','high','low','close','vol','close_time','quote_asset_vol','number_of_trades','taker_base_asset_vol','taker_quote_asset_vol','ignore']
            df=pd.read_csv(file_name,header=0,names=hData,parse_dates=True)
            df['open_time'] = [datetime.fromtimestamp(x/1000) for x in df['open_time']]
            df['close_time'] = [datetime.fromtimestamp(x/1000) for x in df['close_time']]
            df['ticker']=ticker
            df.set_index(pd.DatetimeIndex(df['open_time']), inplace=True)
       
            s3_file=s3_path+'/'+x+'.snappy.parquet'
            print("upload to %s" % s3_file)
            r=wr.s3.to_parquet(
                    df=df,
                    path=s3_file,
                    dataset=False
            )

importFiles('btc','BTCUSDT')
# importFiles('eth','ETHUSDT')